In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_cnn, hyspec_io

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Paths
json_gray = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_20230228_gray/label_classes.json')
annotation_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_20230228_gray/OlbergAreaS')
hyspec_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4_Rad_Georef_SGC')
tiles_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles')

In [5]:
# Parameters
tile_shape = (128,128)

# Instructions for merging / extracting classes
classes_to_merge = [['Zostera marina - dominant','Zostera marina - normal','Zostera marina - sparse'],
                    ['Rockweed','Rockweed with turf algae','Fucus serratus','Saccharina latissima'],
                    ['Turf algae - dominant','Turf algae - normal','Turf algae - sparse','Aegagropila (algae ball)']]
merged_class_names = ['Zostera marina',
                      'Rockweed',
                      'Other algae']
classes_to_extract = ['Sand',
                      'Zostera marina',
                      'Zostera marina with turf algae',
                      'Rockweed',
                      'Other algae',
                      'Zostera marina - NGT',
                      'Rockweed - NGT',
                      'Other algae - NGT']

In [6]:
# Read annotation metadata file, show classes
class_dict = annotation.read_hasty_metadata(json_gray)

In [7]:
ann_files = misc.file_pattern_search(str(annotation_dir),'*.png')
for file in tqdm.tqdm(ann_files):
    # Construct input/output paths for current file
    file_stem = pathlib.Path(file).stem
    png_path = annotation_dir / (file_stem + '.png')
    hyspec_path = hyspec_dir / (file_stem + '.bip.hdr')
    tiles_path = tiles_dir / file_stem
    
    # Read hyperspectral image
    hyspec_im,_,_,_ = hyspec_io.load_envi_image(str(hyspec_path))
    nonzero_mask = np.any(hyspec_im!=0,axis=2)
    
    # Read and filter image (class mask)
    class_mask = skimage.io.imread(str(png_path)) * nonzero_mask
    merged_class_dict, merged_class_mask = annotation.merge_classes_with_mask(
        class_dict,class_mask,classes_to_merge,merged_class_names)
    filtered_class_dict, filtered_class_mask = annotation.extract_subset(
        merged_class_dict,merged_class_mask,classes_to_extract)
    
    # Extract tiles
    tiles = hyspec_cnn.labeled_image_to_tensor_tiles(hyspec_im,filtered_class_mask,tile_shape)
    
    # Create dataset and save as file(s)
    dataset = tf.data.Dataset.from_tensor_slices(tiles)
    tf.data.experimental.save(dataset,str(tiles_path))
    
# Save "filtered" class dict 
filtered_class_dict_path = tiles_dir / 'tile_classes.json'
annotation.save_class_dict(filtered_class_dict,str(filtered_class_dict_path))

100%|████████████████████████████████████████████████████████| 18/18 [03:43<00:00, 12.41s/it]
